In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
import time
import re
import os

In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}

In [ ]:
# tensroflow hub module for Universal sentence Encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3" 
#@param ["https://tfhub.dev/google/universal-sentence-encoder/2", 
#"https://tfhub.dev/google/universal-sentence-encoder-large/3"]
start = time.time()
embed = hub.Module(module_url)
end = time.time()
print(end-start)

In [ ]:
path  = "path/to/text_file/"
with open(path+"text_filename",'r') as f:
    text  = f.read()
print(f)

In [ ]:
# filename = input_filename+'.txt'
# with open(filename, 'r') as data:
#   tr = data.read()
# tr = tr.replace('\n','')

In [ ]:
file1 = open("results/query_results/66c.txt","w") 

In [ ]:
def splitParagraphIntoSentences(paragraph):
    ''' break a paragraph into sentences
        and return a list '''
    import re
    # to split by multile characters

    #   regular expressions are easiest (and fastest)
    sentenceEnders = re.compile('[!.?]')
    sentenceList = sentenceEnders.split(paragraph)
    return sentenceList

In [ ]:

if __name__ == '__main__':
    p = tr
    nlst = []
    from IPython.display import HTML, display
    import tabulate
    sentences = splitParagraphIntoSentences(p)
    i = 0
    for s in sentences:
        
        #print(i)
        k = str(i) + ") " + s.strip()
        i = i+1
        #nlst.append[k]
#         table  = k
#         display(HTML(tabulate.tabulate(table, tablefmt='html')))
        file1.write("\n") 
            
        file1.writelines(k) 
        file1.write("\n")
        print (k)
#       
#     s.strip()

In [ ]:
#for text embeddings : 
def get_features(texts):
    if type(texts) is str:
        texts = [texts]
    with tf.Session() as sess:
        sess.run([tf.global_variables_initializer(), tf.tables_initializer()])
        return sess.run(embed(texts))

In [ ]:
#preprocessing the text via removing stop words and some other symbols
def remove_stopwords(stop_words, tokens):
    res = []
    for token in tokens:
        if not token in stop_words:
            res.append(token)
    return res

def process_text(text):
    text = text.encode('ascii', errors='ignore').decode()
    text = text.lower()
    text = re.sub(r'http\S+', ' ', text)
    text = re.sub(r'#+', ' ', text )
    text = re.sub(r'@[A-Za-z0-9]+', ' ', text)
    text = re.sub(r"([A-Za-z]+)'s", r"\1 is", text)
    #text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"won't", "will not ", text)
    text = re.sub(r"isn't", "is not ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip()
    return text

def lemmatize(tokens):
    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemma_list = []
    for token in tokens:
        lemma = lemmatizer.lemmatize(token, 'v')
        if lemma == token:
            lemma = lemmatizer.lemmatize(token)
        lemma_list.append(lemma)
    # return [ lemmatizer.lemmatize(token, 'v') for token in tokens ]
    return lemma_list


def process_all(text):
    text = process_text(text)
    return ' '.join(remove_stopwords(stop_words, text.split()))


In [ ]:
data = sentences
data_processed = list(map(process_text, data))

In [ ]:
#creating embedding vectors of the data
BASE_VECTORS = get_features(data)
BASE_VECTORS.shape

In [ ]:
#cosine similarity metric for finding the semantic similarity between the splitted sentences and the phrase/query 
# that is needed to be checked.
def cosine_similarity(v1, v2):
    mag1 = np.linalg.norm(v1)
    mag2 = np.linalg.norm(v2)
    if (not mag1) or (not mag2):
        return 0
    return np.dot(v1, v2) / (mag1 * mag2)

In [ ]:
def test_similarity(text1, text2):
    vec1 = get_features(text1)[0]
    vec2 = get_features(text2)[0]
    print(vec1.shape)
    return cosine_similarity(vec1, vec2)

In [ ]:
def semantic_search(query, data, vectors):
    from IPython.display import HTML, display
    import tabulate
    
    query = process_text(query)
    print("Extracting features...")
    query_vec = get_features(query)[0].ravel()
    res = []
    for i, d in enumerate(data):
        qvec = vectors[i].ravel()
        sim = cosine_similarity(query_vec, qvec)*100
        res.append((sim, d[:100], i))
    sr  = sorted(res, key=lambda x : x[0], reverse=True)
# appending the results in the text file
    file1.write("\n") 
    file1.write("Querry Result :  \n")
    for i in sr:        
        file1.writelines(str(i)) 
        file1.write("\n")
    #file1.close()
    table = sr
#     print(sr,"\n")
#     print()
    df = pd.DataFrame(data=np.array(table), columns=['similarity', 'sentence', 'sent_index'])
    return df.head()
   # return display(HTML(tabulate.tabulate(table, tablefmt='html')))

In [ ]:
query1 = "sentence to be checked in the paragraph/text"

In [ ]:
file1.write("\n") 
file1.write(query1)
print("Query : ",query1,"\n")
semantic_search(query1, data_processed, BASE_VECTORS)